# Keysight 33512B Daisy-Chain Demo

This notebook chains API calls using `MockTransport` so you can validate command flow before connecting real hardware.

In [ ]:
import os
import sys

cwd = os.getcwd()
if cwd.endswith(os.path.join('Instruments', 'keysight_33512b')):
    sys.path.insert(0, os.path.abspath(os.path.join(cwd, '..', '..')))
else:
    sys.path.insert(0, os.path.abspath(cwd))

from Instruments.keysight_33512b import Keysight33512B, MockTransport


In [ ]:
def build_driver():
    transport = MockTransport()
    transport.set_query_response('*IDN?', 'KEYSIGHT,33512B,MY12345678,1.00')
    transport.set_query_response('SYST:ERR?', '+0,"No error"')
    awg = Keysight33512B(transport)
    return awg, transport


def expect_raises(exc_type, fn):
    try:
        fn()
    except exc_type:
        return True
    return False


In [ ]:
# Test 1: core SCPI operations
awg, transport = build_driver()

assert awg.identify().startswith('KEYSIGHT,33512B')
awg.clear_status()
awg.reset()
assert awg.get_error().startswith('+0')

assert transport.writes == ['*CLS', '*RST']
print('PASS: core SCPI operations')


In [ ]:
# Test 2: daisy-chain output configuration
awg, transport = build_driver()

awg.set_waveform('SIN', channel=1)
awg.set_frequency_hz(1000.0, channel=1)
awg.set_amplitude_vpp(2.0, channel=1)
awg.set_offset_v(0.1, channel=1)
awg.set_phase_deg(45.0, channel=1)
awg.set_output(True, channel=1)

expected = [
    'SOUR1:FUNC SIN',
    'SOUR1:FREQ 1000.0',
    'SOUR1:VOLT 2.0',
    'SOUR1:VOLT:OFFS 0.1',
    'SOUR1:PHAS 45.0',
    'OUTP1 ON',
]
assert transport.writes == expected
print('PASS: daisy-chain command flow')
print(transport.writes)


In [ ]:
# Test 3: channel 2 check
awg, transport = build_driver()

awg.set_waveform('SQU', channel=2)
awg.set_frequency_hz(1234.5, channel=2)
awg.set_output(False, channel=2)

assert transport.writes == [
    'SOUR2:FUNC SQU',
    'SOUR2:FREQ 1234.5',
    'OUTP2 OFF',
]
print('PASS: channel 2 flow')


In [ ]:
# Test 4: validation behavior
awg, _ = build_driver()

assert expect_raises(ValueError, lambda: awg.set_frequency_hz(0.0))
assert expect_raises(ValueError, lambda: awg.set_amplitude_vpp(-1.0))
assert expect_raises(ValueError, lambda: awg.set_output(True, channel=3))
assert expect_raises(ValueError, lambda: awg.set_waveform(''))
print('PASS: validation checks')
